<a href="https://www.kaggle.com/code/assemafanah/assem-house-price-1?scriptVersionId=117313454" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
#load data sets
train_data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv", index_col=0)
#print("Summary of the DataFrame:");
#uncomment to check data
#print(train_data.info());
#train_data.head()
#-------------------------------------------------------------------------------#
test_data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv", index_col=0)
#check
#test_data.head()

In [3]:
#import libraries
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [4]:
#drop columns with mostly NAN and the catorigal featuers with high cardinality
drop_list = ['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature', 'Neighborhood', 'Exterior1st', 'Exterior2nd']
train_data_drop = train_data.drop(drop_list, axis=1)
# check
train_data_drop.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [5]:
#similar for test data
test_data_drop = test_data.drop(drop_list, axis=1)
# check
test_data_drop.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,120,0,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,36,0,0,0,0,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,34,0,0,0,0,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,36,0,0,0,0,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,Gtl,...,82,0,0,144,0,0,1,2010,WD,Normal


In [6]:
#preprocicssing of data
target = 'SalePrice'
Y = train_data_drop[target]
X = train_data_drop.drop([target], axis=1)
#seperate numerical and caterogical featuers
X_num = X.select_dtypes(include=np.number).copy()
X_cat = X.select_dtypes(include= 'object').copy()
test_num = test_data_drop.select_dtypes(include=np.number).copy()
test_cat = test_data_drop.select_dtypes(include= 'object').copy()
#----------------------------------------------------------------------------------#
#replace missing vlues with mean(numeric) or most recuurent(caterogical)
X_num = X_num.fillna(X_num.mean())
test_num = test_num.fillna(test_num.mean())
X_cat = X_cat.apply(lambda x: x.fillna(x.value_counts().index[0]))
test_cat = test_cat.apply(lambda x: x.fillna(x.value_counts().index[0]))
#-----------------------------------------------------------------------------------#
# standarize numerical features
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_num)
test_num_scaled = scaler.fit_transform(test_num)
#----------------------------------------------------------------------------------#
# Apply ordinal encoder to catoricical featuers  
ordinal_encoder = OrdinalEncoder()
X_cat_OE = ordinal_encoder.fit_transform(X_cat)
test_cat_OE = ordinal_encoder.fit_transform(test_cat)
#---------------------------------------------------------------------------------#
#check
print(X_num_scaled.shape)
print(X_cat_OE.shape)

(1460, 36)
(1460, 35)


In [7]:
# joint datasets
X_num_cat = np.concatenate((X_num_scaled, X_cat_OE), axis=1)
test_num_cat = np.concatenate((test_num_scaled, test_cat_OE), axis=1)
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X_num_cat, Y, train_size=0.8,test_size=0.2, random_state=1)

In [8]:
#Apply recursive feature selection
estimator = RandomForestRegressor(n_estimators=100, max_depth=20)
rfe = RFE(estimator= estimator, n_features_to_select=25, step=1)
rfe.fit(train_X, train_y)
#check validation set predictions
val_predict = rfe.predict(val_X)
print(mean_absolute_error(val_predict, val_y))

17091.905685699578


In [9]:
predictions = rfe.predict(test_num_cat)
output = pd.DataFrame({"Id":test_data_drop.index, target:predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


Different tests and checks of numerical and catorigical featuers 
to choose either to impute data or remove missing values : 
1. numerical features : check nan countes and statistical description.
2. catorigical features : check cardinalty and nan values.   

In [10]:
"""
# choose numeric features only
train_data_num = train_data.select_dtypes(include=np.number).copy()
features_num = list(train_data_num.columns)
print('dimensions of training data_num = ', train_data_num.shape)
print('dimensions of test data = ', test_data_num.shape)
print(features_num)
train_data_num.head()
"""

"\n# choose numeric features only\ntrain_data_num = train_data.select_dtypes(include=np.number).copy()\nfeatures_num = list(train_data_num.columns)\nprint('dimensions of training data_num = ', train_data_num.shape)\nprint('dimensions of test data = ', test_data_num.shape)\nprint(features_num)\ntrain_data_num.head()\n"

In [11]:
"""
# check for columns with NAN
count_nan = train_data_num.isnull().sum()
print(count_nan [count_nan >0])
"""

'\n# check for columns with NAN\ncount_nan = train_data_num.isnull().sum()\nprint(count_nan [count_nan >0])\n'

In [12]:
"""
#drop rows with missing values
train_data_num_drop = train_data_num.dropna(axis=0).reset_index(drop=True)
print(train_data_num_drop.shape)
train_data_num_drop.head()
"""

'\n#drop rows with missing values\ntrain_data_num_drop = train_data_num.dropna(axis=0).reset_index(drop=True)\nprint(train_data_num_drop.shape)\ntrain_data_num_drop.head()\n'

In [13]:
"""
#stat_properties of data set
train_data_num_drop.describe()
"""

'\n#stat_properties of data set\ntrain_data_num_drop.describe()\n'

In [14]:
"""
# catorigical features
train_data_cat = train_data.select_dtypes(include= 'object').copy()
features_cat = list(train_data_cat.columns)
print('dimensions of training data_cat = ', train_data_cat.shape)
train_data_cat.head()
"""

"\n# catorigical features\ntrain_data_cat = train_data.select_dtypes(include= 'object').copy()\nfeatures_cat = list(train_data_cat.columns)\nprint('dimensions of training data_cat = ', train_data_cat.shape)\ntrain_data_cat.head()\n"

In [15]:
"""
# check for columns with NAN
count_nan = train_data_cat.isnull().sum()
print(count_nan [count_nan >100])
print(count_nan [(count_nan>0) & (count_nan<100)])
"""

'\n# check for columns with NAN\ncount_nan = train_data_cat.isnull().sum()\nprint(count_nan [count_nan >100])\nprint(count_nan [(count_nan>0) & (count_nan<100)])\n'

In [16]:
"""
# drop columns with mostly NAN and rows with with missing values
drop_list = ['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature', 'Neighborhood', 'Exterior1st', 'Exterior2nd']
train_data_cat_drop = train_data_cat.drop(drop_list, axis=1).reset_index(drop=True)
#train_data_cat_drop = train_data_cat_drop.dropna(axis=0).reset_index(drop=True)
print(train_data_cat_drop.shape)
# invistigate cardinality
count_uniq = train_data_cat_drop.nunique()
print(count_uniq)
"""

"\n# drop columns with mostly NAN and rows with with missing values\ndrop_list = ['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature', 'Neighborhood', 'Exterior1st', 'Exterior2nd']\ntrain_data_cat_drop = train_data_cat.drop(drop_list, axis=1).reset_index(drop=True)\n#train_data_cat_drop = train_data_cat_drop.dropna(axis=0).reset_index(drop=True)\nprint(train_data_cat_drop.shape)\n# invistigate cardinality\ncount_uniq = train_data_cat_drop.nunique()\nprint(count_uniq)\n"

In [17]:
""" sns.scatterplot(x= train_data['Neighborhood'], y= train_data['SalePrice'])"""

" sns.scatterplot(x= train_data['Neighborhood'], y= train_data['SalePrice'])"